# penjelaskan Naive Bayes

Algoritma Naive Bayes (NB) adalah metode yang menggunakan prinsip probability utk membuat model prediksi klasifikasi.

Keunggulan NB adalah sifatnya yg efektif dan cepat utk mengolalh data berjumblah besar.

## Bagaimana Naive Bayes Bekerja

NB adalah sekumpulan (bukan hanya satu) algoritma klasifikasi yg dibangun berdasarkan Teori Bayes.

Model klasifikasi yg memakai metode bayes memanfaatkan training dataset utk menghitung kemungkinan setiap kelas berdasarkan nilai-nilai feature di dalamnya.

conditional probability yaitu kemungkinan munculnya suatu kejadian A bila suatu kejadian B muncul. Karena kejadian A bergantung kpd kejadian B, itu sebabnya disebut conditional(bersyarat).

Posterior probability (probabilitas posterior) adalah probabilitas suatu kejadian atau hipotesis setelah memiliki informasi baru atau bukti yang dapat mempengaruhi probabilitas tersebut. Dalam statistika dan teori probabilitas, posterior probability diperoleh dengan menggunakan Teorema Bayes, yaitu sebuah formula matematika yang menghubungkan probabilitas posterior dengan probabilitas prior (probabilitas sebelumnya) dan likelihood (kemungkinan suatu data di bawah hipotesis tertentu).

Secara sederhana, posterior probability dapat diartikan sebagai "probabilitas suatu hipotesis setelah melihat data baru". Misalnya, kita ingin mengetahui apakah seseorang menderita penyakit tertentu berdasarkan gejalanya. Probabilitas priornya (sebelum melihat gejala) mungkin rendah, tetapi setelah melihat gejala yang muncul, probabilitas posteriornya akan meningkat atau berubah sesuai dengan informasi baru tersebut.

# Contoh Kasus Kapal Titanic

kita akan mencoba membuat model prediksi apakah seorang penumpang kapal Titanic akan selamat atau tdk dalam kecelakaan tragis ini.

## import

In [1]:
import pandas as pd

## train

In [2]:
df_train = pd.read_csv("titanic_train.csv")
df_train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Feature | Penjelasan
------|-----
pasengerID | Nomer urut penumpang
Name | Nama Penumpang
Pclass | Tiket kleas kabin yg dibeli penumpang(kelas 1,2 atau 3)
Sex | Jenis Kelamin penumpang
Age | Usia penumpang saat kecelakan terjadi
SibSp | Jumlah kerabat atau pasangan yang ikut serta
Parch | Jumlah orang tua atau anak ikut serta
Ticket | nomer ticket penumpang
Fare | Harga tiket yg dibeli penumpang
cabin | Nomer kabin penumpang
Embarked | Nama kota pelabuan asal (C = Cherbourg; Q = Queenstown; S = Southampton)
Survived | Selamat (1) atau Tidak Selamat (0)


Karena kita akan mencoba membuat model prediksi apakah seseorang akan selamat atau tidak, maka kolom "Survived" akan kita jadikan label atau target feature dalam training dataset ini. Sisanya sebelas kolom akan kita pilih mana saja yang layak menjadi feature untuk proses training model.

In [3]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Pertama-tama kita lihat berapa banyak yg memiliki nilai null(kosong) : 

In [4]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

karna cabin terdapat 687 yg kosong maka informasi ini tidak berguna utk proses prediksi

In [5]:
df_train.drop(['Cabin'], axis=1, inplace = True)

In [7]:
# value_counts() Berguna untuk menghitung berapa kali sebuah value terdapat dalam dataset
df_train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
# fFungsi fillna() digunakan untuk mengisi nilai NA/NaN menggunakan metode yang ditentukan.
df_train['Embarked'].fillna('S', inplace = True)